# Import packages


In [ ]:
!pip install --upgrade nomic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install striprtf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nomic
from nomic import atlas
nomic.login('aXDsu14Splh3lVXgHBojThmCQ1Q_wMO4DkXY4gp0IjEtx') #api key to a limited demo account. Make your own account at atlas.nomic.ai

In [ ]:
from nomic import AtlasProject

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
import string
from striprtf.striprtf import rtf_to_text

# Data proprocessing


### Functions

In [ ]:
def get_files(root_dir):
  # Define a list to store the text
  file_list = []

  # Recursively search for files in the root directory and its subdirectories
  for subdir, dirs, files in os.walk(root_dir):
      for fil in files:
          obj = {}
          file_path = os.path.join(subdir, fil)
          seg = subdir.split('/')
          obj['term'] = seg[-1]
          f = open(file_path, 'r')
          text = f.read()
          obj['text'] = text
          f.close
          file_list.append(obj)
  return file_list

In [ ]:
def extract_doc (file_list):
  regex_pattern = "(.*?)Database:"
  doc_list = []
  for obj in file_list:
    matches = re.finditer(regex_pattern, obj['text'], re.DOTALL)
    for mat in matches:
      doc = {}
      extracted_text = mat.group(1).strip()
      doc['text'] = extracted_text
      doc['term'] = obj['term']
      doc_list.append(doc)
  return doc_list

In [ ]:
def documentation(doc_list):
  cont_list = []
  pattern0 = r"Full text:(.*?)Location:"
  pattern1 = r"document ID:(.*?)\n"
  pattern2 = r"Source type:(.*?)\n"
  pattern3 = r"Publication date:(.*?)\n"
  pattern4 = r"^.*?Subject:.*?$"
  pattern5 = r"^.*?Business indexing term:.*?$"
  source = "NA"
  date = "NA"
  for doc in doc_list:
    t = {}
    m0 = re.search(pattern0, doc['text'], re.DOTALL)
    cont = m0.group(1).strip()
    m1 = re.search(pattern1, doc['text'], re.DOTALL)
    id = m1.group(1).strip()
    m2 = re.search(pattern2, doc['text'], re.DOTALL)
    if m2:
      source = m2.group(1).strip()
    m3 = re.search(pattern3, doc['text'], re.DOTALL)
    if m3:
      date = m3.group(1).strip()
    m4 = re.search(pattern4, cont, re.DOTALL)
    m5 = re.search(pattern5, cont, re.DOTALL)
    if m5:
      cont = re.sub(pattern5, "", cont, flags=re.MULTILINE)
    if m4:
      cont = re.sub(pattern4, "", cont, flags=re.MULTILINE)
    t['term'] = doc['term']
    t['full_text'] = cont
    t['date'] = date
    t['source'] = source
    t['id'] = id
    cont_list.append(t)
  return cont_list



In [ ]:
def remove_links(text):
    pattern = r'\( https?://[^\s()]+ \)'
    text_without_links = re.sub(pattern, '', text)
    return text_without_links

In [ ]:
def rm_duplicates(cont_list):
  unique_data = []
  seen_texts = set()
  for entry in cont_list:
      full_text = entry['full_text']
      if full_text not in seen_texts:
          unique_data.append(entry)
          seen_texts.add(full_text)
  return unique_data

In [ ]:
data = {
    "text": "This is a sample text with a link ( https://example.com ). Please click here ( https://google.com ) for more information."
}

# Remove links from the "text" key
data["text"] = remove_links(data["text"])

# Print the updated text
print(data["text"])

This is a sample text with a link . Please click here  for more information.


### Import data

In [ ]:
root_dir = '/content/drive/MyDrive/audace_trial/corpus/english_modern'

In [ ]:
# Define a list to store the text from .rtf files
text_list = []

# Recursively search for files in the root directory and its subdirectories
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        print(file_path)
        with open(file_path, 'r') as f:
            text = f.read()
            text_list.append(text)

/content/drive/MyDrive/audace_trial/corpus/english_modern/freezingrain/ProQuest_freezingrain_20230513(1).txt
/content/drive/MyDrive/audace_trial/corpus/english_modern/freezingrain/ProQuest_freezingrain_20230513(2).txt
/content/drive/MyDrive/audace_trial/corpus/english_modern/flood/flood_gazette-2023-05-12.txt
/content/drive/MyDrive/audace_trial/corpus/english_modern/forestfire/ProQuest_forestfire_20230513.txt
/content/drive/MyDrive/audace_trial/corpus/english_modern/blizzard/ProQuest_Blizzard_20230513.txt
/content/drive/MyDrive/audace_trial/corpus/english_modern/heatwave/ProQuest_heatwave_20230513(2).txt
/content/drive/MyDrive/audace_trial/corpus/english_modern/heatwave/ProQuest_heatwave_20230513(1).txt


### Data cleaning

In [ ]:
root_dir = '/content/drive/MyDrive/audace_trial/corpus/english_modern'

In [ ]:
file_list = get_files(root_dir)

In [ ]:
doc_list = extract_doc(file_list)

In [ ]:
cont_list = documentation(doc_list)

In [ ]:
unique_list = rm_duplicates(cont_list)

In [ ]:
dict_list = [{'text': string['full_text']} for string in unique_list]

In [ ]:
for i in range(len(dict_list)):
    dict_list[i]['id'] = i

In [ ]:
dict_list

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for item in dict_list:
    item["text"] = remove_links(item["text"])

In [ ]:
dict_list

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
english_modern_project = atlas.map_text(data=dict_list,
                         id_field='id',
                         indexed_field='text',
                         name='English_Modern_Proquest_06_07_2023',
                         description='A classification of english_modern files. (Proquest; 2000-2020)'
                         )

2023-06-08 00:14:29.933 | INFO     | nomic.project:_create_project:965 - Creating project `English_Modern_Proquest_06_07_2023` in organization `wangmc331`
2023-06-08 00:14:31.288 | INFO     | nomic.atlas:map_text:212 - Uploading text to Atlas.
2023-06-08 00:14:31.402 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:244 - id_field is not a string. Converting to string from int32
3it [00:02,  1.01it/s]
2023-06-08 00:14:34.385 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-06-08 00:14:34.389 | INFO     | nomic.atlas:map_text:228 - Text upload succeeded.
2023-06-08 00:14:35.443 | INFO     | nomic.project:create_index:1282 - Created map `English_Modern_Proquest_06_07_2023` in project `English_Modern_Proquest_06_07_2023`: https://atlas.nomic.ai/map/02bdebe8-92c9-4470-a840-ed3b876f32e1/202f0a8d-0552-4e04-89cd-eb5936d65cf3
2023-06-08 00:14:35.445 | INFO     | nomic.atlas:map_text:242 - English_Modern_Proquest_06_07_2023: https://atlas.nomic.ai/map/02bdebe8-92c9

In [ ]:
english_modern_project.maps[0]

English_Modern_Proquest_06_07_2023: https://atlas.nomic.ai/map/02bdebe8-92c9-4470-a840-ed3b876f32e1/202f0a8d-0552-4e04-89cd-eb5936d65cf3

In [ ]:
english_modern_project_tm = atlas.map_text(data=dict_list,
                         id_field='id',
                         indexed_field='text',
                         name='English_Modern_topic_model_Proquest_06_07_2023',
                         description='A classification of english_modern files. (Proquest; 2000-2020)',
                         build_topic_model = True
                         )

2023-06-08 00:15:52.795 | INFO     | nomic.project:_create_project:965 - Creating project `English_Modern_topic_model_Proquest_06_07_2023` in organization `wangmc331`


Exception: ignored

In [ ]:
english_modern_project_tm.maps[0]

English_Modern_topic_model_Proquest: https://atlas.nomic.ai/map/3dbdd6d0-a9ee-44c0-979e-3d7f0fd306bc/3af8126b-7481-4e25-9f2b-820acd229af4

# Project

In [ ]:
project = AtlasProject(name='English_Modern_Proquest')
map = project.maps[0]

2023-06-07 19:48:49.068 | INFO     | nomic.project:__init__:882 - Loading existing project `English_Modern_Proquest` from organization `wangmc331`.


In [ ]:
tags = map.get_tags()
for tag, datum_ids in tags.items():
    print(tag, "Count:", len(datum_ids), datum_ids[:10])

print(project.get_data(ids=tags['sport'][:2]))

politics Count: 44 ['1154', '118', '1243', '1256', '1257', '1258', '1464', '1583', '1584', '1686']
sport Count: 72 ['1246', '1247', '1248', '1249', '1250', '1251', '1252', '1253', '1421', '1422']
[{'id': '1246', 'text': 'Canadiens Game Day is a new feature this season in which we will follow the Habs during every home game from the morning skate in Brossard through the post-game interviews at the Bell Centre, updating throughout the day with all the news, quotes, notes and opinion you\'re looking for:Former Canadien Alexander Radulov didn\'t want to talk to the media who showed up at the Bell Centre for the Dallas Stars\' morning skate on Tuesday, walking past them and simply saying: "No."He still didn\'t want to talk to the Montreal media after the Stars\' 4-1 win over the Canadiens Tuesday night in which he scored the final goal into an empty net.A Stars media-relations person said Radulov was unavailable because he was undergoing post-game treatment. When J.F. Chaumont of the Journa